In [1]:
%load_ext autoreload
%autoreload 2

# `Logit` on Orders - A warm-up challenge (~1h)

## Select features

🎯 Let's figure out the impact of `wait_time` and `delay_vs_expected` on very `good/bad reviews`

👉 Using our `orders` training_set, we will run two `multivariate logistic regressions`:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

 

In [2]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Import your dataset:

In [3]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

/Users/fercancercel/code/fercanc/data-context-and-setup/olist/order.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
/Users/fercancercel/code/fercanc/data-context-and-setup/olist/order.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
/Users/fercancercel/code/fercanc/data-context-and-setup/olist/order.py:33: SettingWi

👉 Select in a list which features you want to use:

⚠️ Make sure you are not creating data leakage (i.e. selecting features that are derived from the target)

💡 To figure out the impact of `wait_time` and `delay_vs_expected` we need to control for the impact of other features, include in your list all features that may be relevant

In [20]:
# YOUR CODE HERE
values = orders[["order_id","wait_time","delay_vs_expected", "review_score","number_of_sellers","price","freight_value"]]
values

,order_id,wait_time,delay_vs_expected,review_score,number_of_sellers,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,0.0,4,1,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,0.0,4,1,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,0.0,5,1,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,0.0,5,1,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,0.0,5,1,19.90,8.72
...,...,...,...,...,...,...,...
95875,9c5dedf39a927c1b2549525ed64a053c,8.218009,0.0,5,1,72.00,13.08
95876,63943bddc261676b46f01ca7ac2f7bd8,22.193727,0.0,4,1,174.90,20.10
95877,83c1379a015df1e13d02aae0204711ab,24.859421,0.0,5,1,205.99,65.02
95878,11c177c8e97725db2631073c19f07b62,17.086424,0.0,2,1,359.98,81.18


🕵🏻 Check the `multi-colinearity` of your features, using the `VIF index`.

* It shouldn't be too high (< 10 preferably) to ensure that we can trust the partial regression coefficents and their associated `p-values` 
* Do not forget to standardize your data ! 
    * A `VIF Analysis` is made by regressing a feature vs. the other features...
    * So you want to `remove the effect of scale` so that your features have an equal importance before running any linear regression!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardizing:

In [47]:
# YOUR CODE HERE
columns=["wait_time","delay_vs_expected", "review_score","number_of_sellers","price","freight_value"]

for f in columns:
    mean=values[f].mean() 
    std=values[f].std()   
    values[f]=values[f].map(lambda x: (x-mean)/std)
    
values["dim_is_one_star"] = orders['dim_is_one_star']
values["dim_is_five_star"] = orders["dim_is_five_star"]
values

/var/folders/kg/6qqrzjg91x5d3tq16qx18m2w0000gn/T/ipykernel_4237/1404154253.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values[f]=values[f].map(lambda x: (x-mean)/std)
/var/folders/kg/6qqrzjg91x5d3tq16qx18m2w0000gn/T/ipykernel_4237/1404154253.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values["dim_is_one_star"] = orders['dim_is_one_star']
/var/folders/kg/6qqrzjg91x5d3tq16qx18m2w0000gn/T/ipykernel_4237/1404154253.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

,wait_time,delay_vs_expected,review_score,number_of_sellers,price,freight_value,dim_is_one_star,dim_is_five_star
0,-0.431192,-0.161781,-0.121008,-0.112544,-0.513802,-0.652038,0,0
1,0.134174,-0.161781,-0.121008,-0.112544,-0.086640,0.000467,0,0
2,-0.329907,-0.161781,0.657133,-0.112544,0.111748,-0.164053,0,1
3,0.073540,-0.161781,0.657133,-0.112544,-0.441525,0.206815,0,1
4,-1.019535,-0.161781,0.657133,-0.112544,-0.562388,-0.652038,0,1
...,...,...,...,...,...,...,...,...
95875,-0.454309,-0.161781,0.657133,-0.112544,-0.311513,-0.449408,0,1
95876,1.023841,-0.161781,-0.121008,-0.112544,0.183977,-0.123156,0,0
95877,1.305780,-0.161781,0.657133,-0.112544,0.333684,1.964490,0,1
95878,0.483664,-0.161781,-1.677291,-0.112544,1.075186,2.715522,0,0


👉 Run your VIF Analysis to analyze the potential multicolinearities:

In [48]:
# YOUR CODE HERE

from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(values.values,i) for i in range(values.shape[1])]
vif


,VIF
0,2.171700
1,2.037291
2,6.520806
3,1.042730
4,1.204060
5,1.280761
6,4.832550
7,2.647485


## Logistic Regressions

👉 Fit two `Logistic Regression` models:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

`Logit 1️⃣`

In [63]:
# YOUR CODE HERE

logit_one = smf.logit(formula='dim_is_one_star ~ wait_time + delay_vs_expected', data=values).fit()
logit_one.summary()

Optimization terminated successfully.
         Current function value: 0.283196
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95869
Method:                           MLE   Df Model:                            2
Date:                Thu, 27 Oct 2022   Pseudo R-squ.:                  0.1147
Time:                        12:20:07   Log-Likelihood:                -27151.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -2.3990      0.012   -193.843      0.000      -2.423      -2.375
wait_time             0.5589      0.015     38.510      0.000       0.530       0.587
delay_vs_expected     0.3367      0.018     19.082      0.000       0.302       0.371
=====================================================================================
"""

`Logit 5️⃣`

In [62]:
# YOUR CODE HERE

logit_five = smf.logit(formula='dim_is_five_star ~ wait_time + delay_vs_expected', data=values).fit()
logit_five.summary()

Optimization terminated successfully.
         Current function value: 0.642678
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       dim_is_five_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95869
Method:                           MLE   Df Model:                            2
Date:                Thu, 27 Oct 2022   Pseudo R-squ.:                 0.04941
Time:                        12:19:56   Log-Likelihood:                -61615.
converged:                       True   LL-Null:                       -64817.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.3367      0.007     47.258      0.000       0.323       0.351
wait_time            -0.4406      0.010    -44.065      0.000      -0.460      -0.421
delay_vs_expected    -0.4926      0.023    -21.326      0.000      -0.538      -0.447
=====================================================================================
"""

💡 It's time to analyse the results of these two logistic regressions:

- Interpret the partial coefficients in your own words.
- Check their statistical significances with `p-values`
- Do you notice any differences between `logit_one` and `logit_five` in terms of coefficient importances?

In [59]:
# Among the following sentences, store the ones that are true in the list below

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more more than one_star"

your_answer = [a]

🧪 __Test your code__

In [60]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/fercancercel/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/fercancercel/code/fercanc/data-logit/tests
plugins: anyio-3.6.1, asyncio-0.19.0
asyncio: mode=strict
collecting ... collected 1 item

test_logit.py::TestLogit::test_question PASSED                           [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/logit.pickle

git commit -m 'Completed logit step'

git push origin master



<details>
    <summary>- <i>Explanations and advanced concepts </i> -</summary>


> _All other thing being equal, the `delay factor` tends to increase the chances of getting stripped of the 5-star even more so than it affect the chances of 1-star reviews. Probably because 1-stars are really targeting bad products themselves, not bad deliveries_
    
❗️ However, to be totally rigorous, we have to be **more careful when comparing coefficients from two different models**, because **they might not be based on similar populations**!
    We have 2 sub-populations here: (people who gave 1-stars; and people who gave 5-stars) and they may exhibit intrinsically different behavior patterns. It may well be that "happy-people" (who tends to give 5-stars easily) are less sensitive as "grumpy-people" (who shoot 1-stars like Lucky-Luke), when it comes to "delay", or "price"...

</details>


## Logistic vs. Linear ?

👉 Compare:
- the regression coefficients obtained from the `Logistic Regression `
- with the regression coefficients obtained through a `Linear Regression` 
- on `review_score`, using the same features. 

⚠️ Check that both sets of coefficients  tell  "the same story".

> YOUR ANSWER HERE

In [74]:
# YOUR CODE HERE
model2 = smf.ols(formula='dim_is_one_star ~ wait_time + delay_vs_expected', data=values).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        dim_is_one_star   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.103
Method:                 Least Squares   F-statistic:                     5508.
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:18:24   Log-Likelihood:                -14381.
No. Observations:               95872   AIC:                         2.877e+04
Df Residuals:                   95869   BIC:                         2.880e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0977      0.001    107.549      0.000       0.096       0.099
wait_time             0.0619      0.001     48.491      0.000       0.059       0.064
delay_vs_expected     0.0410      0.001     32.164      0.000       0.039       0.044
==============================================================================
Omnibus:                    46556.109   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           273634.222
Skew:                           2.328   Prob(JB):                         0.00
Kurtosis:                       9.843   Cond. No.                         2.39
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

🏁 Congratulations! 

💾 Don't forget to commit and push your `logit.ipynb` notebook !